# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,18.23,94,93,3.13,NZ,1705661516
1,1,lerwick,60.1545,-1.1494,3.00,94,100,1.31,GB,1705661371
2,2,guanica,17.9716,-66.9080,25.03,49,11,4.71,PR,1705661517
3,3,san fernando de monte cristi,19.8667,-71.6500,22.04,84,0,4.87,DO,1705661517
4,4,sur,22.5667,59.5289,25.56,59,12,4.38,OM,1705661518


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot1=city_data_df.hvplot.points(
    'Lng','Lat',geo=True,tiles='OSM',
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.3,
    color = "City")

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
reduced_city_data_df=city_data_df.loc[(city_data_df['Max Temp']>20) & (city_data_df['Max Temp']<25),:]

# Drop any rows with null values
ideal_weather_city_df=reduced_city_data_df.dropna(how='any',axis=0)

# Display sample data
ideal_weather_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,san fernando de monte cristi,19.8667,-71.6500,22.04,84,0,4.87,DO,1705661517
16,16,suez,29.9737,32.5263,21.34,36,0,3.39,EG,1705661524
24,24,kailua-kona,19.6406,-155.9956,21.98,54,0,4.12,US,1705661529
25,25,dasha,23.1100,113.4444,24.99,54,6,3.70,CN,1705661530
27,27,adamstown,-25.0660,-130.1015,23.99,77,100,7.24,PN,1705661531


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_city_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,san fernando de monte cristi,DO,19.8667,-71.6500,84,
16,suez,EG,29.9737,32.5263,36,
24,kailua-kona,US,19.6406,-155.9956,54,
25,dasha,CN,23.1100,113.4444,54,
27,adamstown,PN,-25.0660,-130.1015,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'
params = {
    "categories":categories,
    "limit": 20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,'Lat']
    longitude=hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter= f"circle:{longitude},{latitude},{radius}"
    bias= f"proximity:{longitude},{latitude}"
    params["filter"] = filter
    params["bias"] = bias 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
san fernando de monte cristi - nearest hotel: No hotel found
suez - nearest hotel: فندق لوسيندا
kailua-kona - nearest hotel: Kona Seaside Hotel
dasha - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
san agustin de valle fertil - nearest hotel: Cabañas Valle Pintado
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
kutum - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
el charco - nearest hotel: No hotel found
darnah - nearest hotel: فندق الفردوس
tyre - nearest hotel: El Boutique Hotel
callao - nearest hotel: Casa Ronald
wanning - nearest hotel: No hotel found
san rafael - nearest hotel: Hotel Regional
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
saint-louis - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
ambatofinandrahana - nearest hotel: No hotel found
sultanah - nearest hotel: فندق جلنار
mizdah -

,City,Country,Lat,Lng,Humidity,Hotel Name
3,san fernando de monte cristi,DO,19.8667,-71.6500,84,No hotel found
16,suez,EG,29.9737,32.5263,36,فندق لوسيندا
24,kailua-kona,US,19.6406,-155.9956,54,Kona Seaside Hotel
25,dasha,CN,23.1100,113.4444,54,No hotel found
27,adamstown,PN,-25.0660,-130.1015,77,No hotel found
...,...,...,...,...,...,...
547,murzuq,LY,25.9155,13.9184,9,فندق باريس - مرزق
553,aldinga beach,AU,-35.2793,138.4604,50,The Beach Hotel
560,kithor,IN,28.8667,77.9333,33,No hotel found
569,reyes,BO,-14.3167,-67.3833,92,Hotel Santa Rosa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [51]:
%%capture --no-display

# Configure the map plot
map_plot= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width= 800,
    frame_height=600,
    size="Humidity",
    scale=1,
    color="r",
    hover_cols=["Hotel Name","Country"]
)
    

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)